In [6]:
#import
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
import scipy.cluster.hierarchy as hr
from geojson import LineString
from geojson import MultiLineString
from shapely.geometry import MultiLineString
import random
import math
import os
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import pairwise_distances
import pandas as pd
import folium
from shapely import wkt, geometry
import json
from pprint import pprint
from openrouteservice import client, places

import dtw_algorithm_edited as dtw

api_key = '5b3ce3597851110001cf62487a4f3ad33ea14791b72150c9e6446e5e'
clnt = client.Client(key=api_key)

In [38]:
#GPS data open and read > path(gps_dummy) generations
x=0
paths ='/Users/cvdg/Downloads/경로예측/코드/gamma/gamma_code_3/Geolife Trajectories 1.3/Data/001'

for filename in os.listdir(paths):
    a=[]
    with open(os.path.join(paths, filename), 'r') as f:
        #print(os.path.join(paths, filename))
        lines = f.readlines()[9:]
        for i in lines:
            c= i.split(sep=',')
            if len(c)>=7:
                b= i.split(sep=',')
                b[6]=b[6].replace("\n","")
                b[6]=b[6].replace(":","")
                a.append(b)
    n_a = np.array(a)
    for i in range(4):
        print(n_a.shape)
        n_a = np.delete(n_a,2,axis=1)
    n_a = n_a.astype(np.float64)
    globals()['GPS_dummy_'+str(x)] = n_a.tolist()
    x+=1

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb7 in position 559: invalid start byte

In [3]:
x=0
for i in range(10):
    paths ='C:/Users/찬란한 돌멩이/Desktop/GPS DAT/Geolife Trajectories 1.3/Geolife Trajectories 1.3/Data/00'+str(i)+'/Trajectory'
    for filename in os.listdir(paths):
        a=[]
        with open(os.path.join(paths, filename), 'r') as f:
            #print(os.path.join(paths, filename))
            lines = f.readlines()[9:]
            for i in lines:
                c= i.split(sep=',')
                if len(c)>=7:
                    b= i.split(sep=',')
                    b[6]=b[6].replace("\n","")
                    b[6]=b[6].replace(":","")
                    a.append(b)
        n_a = np.array(a)
        if n_a.shape[0]>0:
            for i in range(4):
                n_a = np.delete(n_a,2,axis=1)
            n_a = n_a.astype(np.float64)
            globals()['GPS_dummy_'+str(x)] = n_a.tolist()
            x+=1

In [9]:
x

1383

In [4]:
#DTW분석
distance_matrix_dtw=np.zeros(shape=(x,x))
path_list=[]

for i in range(x-1):
    for j in range(i+1,x):
        distance,path = fastdtw(globals()['GPS_dummy_'+str(i)],globals()['GPS_dummy_'+str(j)], dist=dtw.Distance_star)
#normalizing
        distance_matrix_dtw[i][j] = distance/(len(globals()['GPS_dummy_'+str(i)])+len(globals()['GPS_dummy_'+str(j)])-1)

for i in range(x-1):
    for j in range(i+1,x):
                distance_matrix_dtw[i][j] =  (distance_matrix_dtw[i][j]-min(map(min, distance_matrix_dtw)))/(max(map(max, distance_matrix_dtw))-min(map(min, distance_matrix_dtw)))    
distance_matrix_dtw = distance_matrix_dtw.tolist()

KeyboardInterrupt: 

In [39]:
pd.DataFrame(distance_matrix_dtw)

0         1         2         3         4         5         6         7   \
0   0.0  0.311405  0.534158  0.357119  0.468967  0.528372  0.428036  0.012314   
1   0.0  0.000000  0.108711  0.268107  0.146892  0.327770  0.320229  0.058982   
2   0.0  0.000000  0.000000  0.160096  0.047959  0.495354  0.363495  0.020212   
3   0.0  0.000000  0.000000  0.000000  0.109702  0.352434  0.300298  0.069984   
4   0.0  0.000000  0.000000  0.000000  0.000000  0.539502  0.424963  0.034559   
..  ...       ...       ...       ...       ...       ...       ...       ...   
66  0.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
67  0.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
68  0.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
69  0.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
70  0.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

          8         9   ...        61        62        63        64        65  \
0   0.564425  0.482337  ...  0.533829  0.051977  0.578897  0.032396  0.513778   
1   0.331294  0.341831  ...  0.082032  0.149678  0.293512  0.121136  0.343887   
2   0.484655  0.372275  ...  0.277803  0.150895  0.527129  0.045712  0.536762   
3   0.348783  0.313014  ...  0.124819  0.164948  0.347197  0.114962  0.378338   
4   0.544774  0.444983  ...  0.046160  0.127744  0.508017  0.118901  0.555384   
..       ...       ...  ...       ...       ...       ...       ...       ...   
66  0.000000  0.000000  ...  0.000000  0.000000  0.000000  0.000000  0.000000   
67  0.000000  0.000000  ...  0.000000  0.000000  0.000000  0.000000  0.000000   
68  0.000000  0.000000  ...  0.000000  0.000000  0.000000  0.000000  0.000000   
69  0.000000  0.000000  ...  0.000000  0.000000  0.000000  0.000000  0.000000   
70  0.000000  0.000000  ...  0.000000  0.000000  0.000000  0.000000  0.000000   

          66        67        68        69        70  
0   0.243670  0.511996  0.474619  0.513650  0.012232  
1   0.334969  0.340228  0.031344  0.344282  0.065483  
2   0.183231  0.476815  0.021513  0.558447  0.031474  
3   0.284266  0.352242  0.030737  0.385771  0.084811  
4   0.141996  0.519446  0.199381  0.576216  0.051988  
..       ...       ...       ...       ...       ...  
66  0.000000  0.204749  0.023466  0.239628  0.048896  
67  0.000000  0.000000  0.705097  0.442686  0.010070  
68  0.000000  0.000000  0.000000  0.568986  0.033911  
69  0.000000  0.000000  0.000000  0.000000  0.012084  
70  0.000000  0.000000  0.000000  0.000000  0.000000  

[71 rows x 71 columns]

In [77]:
#Agglomerative clustering
n_clusters=10
agg = AgglomerativeClustering(n_clusters= n_clusters, metric='precomputed',
                              linkage='average')
u = agg.fit_predict(distance_matrix_dtw)

In [78]:
#loading Agglomerative Clustering result on differnt lists(클러스터 수만큼의 리스트들)
i_dict={}
a=0
b=0

for i in range(n_clusters):
    globals()['Clusters_'+str(i)]=[]
    temp=[]
    n=0
    for j in u:
        if j == a:
            temp.append(b)
            i_dict[(i,n)]=b
            n+=1
        b+=1
        
    b=0
    globals()['Clusters_'+str(i)] = temp
    a+=1

In [79]:
#initializing_1
for i in range(x):
    globals()['temp_'+str(i)]=globals()['GPS_dummy_'+str(i)]

In [80]:
#initializing_2
for i in range(x):
    globals()['GPS_dummy_'+str(i)]=globals()['temp_'+str(i)]

#based on Clustering results, Making Clustered GPS path lists.
for i in range(n_clusters):
    globals()['Clusters_path_'+str(i)]=[]
    for j in globals()['Clusters_'+str(i)]:
        globals()['GPS_dummy_'+str(j)] = np.array(globals()['GPS_dummy_'+str(j)])
        globals()['GPS_dummy_'+str(j)] = np.delete(globals()['GPS_dummy_'+str(j)],2,1)
        globals()['GPS_dummy_'+str(j)][:,[0,1]] = globals()['GPS_dummy_'+str(j)][:,[1,0]] 
        globals()['Clusters_path_'+str(i)].append(globals()['GPS_dummy_'+str(j)])

In [81]:
#making path polygons[ith_cluster][jth_path]
for i in range(n_clusters):
    for j in range(len(globals()['Clusters_path_'+str(i)])):
        globals()['path_polygon_'+str(i)+str(j)]=[]
        for k in globals()['Clusters_path_'+str(i)][j]:
            globals()['path_polygon_'+str(i)+str(j)].append(str(k[0])+" "+str(k[1])+" ")
        globals()['path_polygon_'+str(i)+str(j)] = ','.join(globals()['path_polygon_'+str(i)+str(j)])
        globals()['path_polygon_'+str(i)+str(j)]= 'LineString'+" "+"("+ globals()['path_polygon_'+str(i)+str(j)] +")"

In [82]:
#mapping all paths
what_cluster=random.randint(0,n_clusters-1)

wkt_str = globals()['path_polygon_'+str(0)+str(0)]
aoi_geom = wkt.loads(wkt_str)

aoi_coords = list(aoi_geom.coords)
aoi_coords = [(y,x) for x,y in aoi_coords] 
aoi_centroid = aoi_geom.centroid 


m = folium.Map(location=(aoi_centroid.y, aoi_centroid.x), zoom_start=16)

for i in range(n_clusters):
    for j in range(len(globals()['Clusters_path_'+str(i)])):
        globals()['wkt_str_'+str(i)+str(j)] = globals()['path_polygon_'+str(i)+str(j)]
        globals()['aoi_geom_'+str(i)+str(j)] = wkt.loads(globals()['wkt_str_'+str(i)+str(j)])
        globals()['aoi_coords_'+str(i)+str(j)] = list(globals()['aoi_geom_'+str(i)+str(j)].coords)
        globals()['aoi_coords_'+str(i)+str(j)] = [(y,x) for x,y in globals()['aoi_coords_'+str(i)+str(j)]] 
        globals()['aoi_centroid_'+str(i)+str(j)]= globals()['aoi_geom_'+str(i)+str(j)].centroid
    
        color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])]

        folium.PolyLine(globals()['aoi_coords_'+str(i)+str(j)],
                                     color=color[0],
                                     #fill_color=color[0],
                                     fill_opacity=0.2,
                                     weight=3).add_to(m)

m

In [94]:
#mapping paths of a cluster randomly chosen
what_cluster=random.randint(0,n_clusters-1)

wkt_str = globals()['path_polygon_'+str(what_cluster)+str(len(globals()['Clusters_path_'+str(what_cluster)])-1)]
aoi_geom = wkt.loads(wkt_str)

aoi_coords = list(aoi_geom.coords)
aoi_coords = [(y,x) for x,y in aoi_coords] 
aoi_centroid = aoi_geom.centroid 


m = folium.Map(location=(aoi_centroid.y, aoi_centroid.x), zoom_start=16)

for i in range(0,len(globals()['Clusters_path_'+str(what_cluster)])):
    globals()['wkt_str_'+str(i)] = globals()['path_polygon_'+str(what_cluster)+str(i)]
    globals()['aoi_geom_'+str(i)] = wkt.loads(globals()['wkt_str_'+str(i)])
    globals()['aoi_coords_'+str(i)] = list(globals()['aoi_geom_'+str(i)].coords)
    globals()['aoi_coords_'+str(i)] = [(y,x) for x,y in globals()['aoi_coords_'+str(i)]] 
    globals()['aoi_centroid'+str(i)]= globals()['aoi_geom_'+str(i)].centroid
    
    color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])]

    folium.PolyLine(globals()['aoi_coords_'+str(i)],
                                     color=color[0],
                                     #fill_color=color[0],
                                     fill_opacity=0.2,
                                     weight=3).add_to(m)
   

m

In [96]:
#extracting all the index numbers of summarized paths of each clusters.
index_for_summarized_path_l=[]
for i in range(n_clusters):
     if len(globals()['Clusters_path_'+str(i)])>2:
          globals()['path_score_count_'+str(i)]=[0 for j in range(len(globals()['Clusters_path_'+str(i)]))]
          for a in range(len(globals()['Clusters_path_'+str(i)])-1):
               for b in range(a+1, len(globals()['Clusters_path_'+str(i)])):
                    distance=distance_matrix_dtw[i_dict[(i,a)]][i_dict[(i,b)]]
                   # distance = (dtw.dtw_pathcost_normalized(globals()['Clusters_path_'+str(i)][a],globals()['Clusters_path_'+str(i)][b]))/(len(globals()['Clusters_path_'+str(i)][a])+len(globals()['Clusters_path_'+str(i)][b])-1)
                    globals()['path_score_count_'+str(i)][a]+=distance/2
                    globals()['path_score_count_'+str(i)][b]+=distance/2
                    
          #index_for_summarized_path[i] = globals()['path_score_count_'+str(i)].index(min(globals()['path_score_count_'+str(i)]))
          index_for_summarized_path_l.append(globals()['path_score_count_'+str(i)].index(min(globals()['path_score_count_'+str(i)])))
               
     if len(globals()['Clusters_path_'+str(i)])==2:
          if len(globals()['Clusters_path_'+str(i)][0]) >= len(globals()['Clusters_path_'+str(i)][1]):
               index_for_summarized_path_l.append(1)
                    
          if len(globals()['Clusters_path_'+str(i)][0]) < len(globals()['Clusters_path_'+str(i)][1]):
               index_for_summarized_path_l.append(0)
                    
                         
     if len(globals()['Clusters_path_'+str(i)])==1:
          index_for_summarized_path_l.append(0)

In [97]:
##making summarized paths for each clusters
for x in range(n_clusters):
    globals()['Summarized_path_of_Clusters_path_'+str(x)]= globals()['Clusters_path_'+str(x)][index_for_summarized_path_l[x]]

In [98]:
#converting a summarized path of clusters into shapely polygon format.
for j in range(n_clusters):    
    globals()['path_polygon_summarized_'+str(j)]=[]
    for i in globals()['Summarized_path_of_Clusters_path_'+str(j)]:
        globals()['path_polygon_summarized_'+str(j)].append(str(i[0])+" "+str(i[1])+" ")
    globals()['path_polygon_summarized_'+str(j)]= ','.join(globals()['path_polygon_summarized_'+str(j)])
    globals()['path_polygon_summarized_'+str(j)]= 'LineString'+" "+"("+globals()['path_polygon_summarized_'+str(j)]+")"

In [99]:
#mapping all summarized paths on openroutemap
what_cluster=random.randint(0,n_clusters-1)

wkt_str = globals()['path_polygon_summarized_'+str(what_cluster)]
aoi_geom = wkt.loads(wkt_str)
aoi_centroid = aoi_geom.centroid 


m = folium.Map(location=(aoi_centroid.y, aoi_centroid.x), zoom_start=16)

for i in range(n_clusters):
    if len(globals()['path_polygon_summarized_'+str(i)])>=20:
        globals()['wkt_str_'+str(i)] = globals()['path_polygon_summarized_'+str(i)]
        globals()['aoi_geom_'+str(i)] = wkt.loads(globals()['wkt_str_'+str(i)])
        globals()['aoi_coords_'+str(i)] = list(globals()['aoi_geom_'+str(i)].coords)
        globals()['aoi_coords_'+str(i)] = [(y,x) for x,y in globals()['aoi_coords_'+str(i)]] 
        globals()['aoi_centroid_'+str(i)]= globals()['aoi_geom_'+str(i)].centroid
        
        color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])]

        folium.PolyLine(globals()['aoi_coords_'+str(i)],
                                        color=color[0],
                                        #fill_color=color[0],
                                        fill_opacity=0.2,
                                        weight=3).add_to(m)
m

In [100]:
#predicting a future path

what_n=random.randint(0,x)
testing_path=globals()['GPS_dummy_'+str(what_n)][:10]
testing_path_whole=globals()['GPS_dummy_'+str(what_n)]

predict_score_count_1=[]
for i in range(n_clusters):
    distance,path = fastdtw(globals()['Summarized_path_of_Clusters_path_'+str(i)],testing_path, dist=dtw.Distance_star)
    predict_score_count_1.append(distance)
y=predict_score_count_1.index(min(predict_score_count_1))

predict_score_count_2=[]
for k in range(len(globals()['Clusters_path_'+str(y)])):
     distance,path = fastdtw(globals()['Clusters_path_'+str(y)][k],testing_path, dist=dtw.Distance_star)
     predict_score_count_2.append(distance)
z=predict_score_count_2.index(min(predict_score_count_2))   

predicted_path_polygon=[]
for i in globals()['Clusters_path_'+str(y)][k]:
    predicted_path_polygon.append(str(i[0])+" "+str(i[1])+" ")
predicted_path_polygon= ','.join(globals()['predicted_path_polygon'])
predicted_path_polygon= 'LineString'+" "+"("+predicted_path_polygon+")"

testing_path_polygon=[]
for i in testing_path:
    testing_path_polygon.append(str(i[0])+" "+str(i[1])+" ")
testing_path_polygon= ','.join(testing_path_polygon)
testing_path_polygon= 'LineString'+" "+"("+testing_path_polygon+")"

testing_path_polygon_whole=[]
for i in testing_path_whole:
   testing_path_polygon_whole.append(str(i[0])+" "+str(i[1])+" ")
testing_path_polygon_whole= ','.join(testing_path_polygon_whole)
testing_path_polygon_whole= 'LineString'+" "+"("+testing_path_polygon_whole+")"



In [107]:
#mapping a testing path and a past path
what_cluster=random.randint(0,n_clusters-1)

wkt_str = globals()['path_polygon_summarized_'+str(what_cluster)]
aoi_geom = wkt.loads(wkt_str)
aoi_centroid = aoi_geom.centroid 


m = folium.Map(location=(aoi_centroid.y, aoi_centroid.x), zoom_start=16)


globals()['wkt_str_p'] = globals()['predicted_path_polygon']
globals()['aoi_geom_p'] = wkt.loads(globals()['wkt_str_p'])
globals()['aoi_coords_p'] = list(globals()['aoi_geom_p'].coords)
globals()['aoi_coords_p'] = [(y,x) for x,y in globals()['aoi_coords_p']] 
globals()['aoi_centroid_p']= globals()['aoi_geom_p'].centroid
        
color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])]
folium.PolyLine(globals()['aoi_coords_p'],
                                        color=color[0],
                                        #fill_color=color[0],
                                        fill_opacity=0.2,
                                        weight=3).add_to(m)

globals()['wkt_str_w'] = testing_path_polygon_whole
globals()['aoi_geom_w'] = wkt.loads(globals()['wkt_str_w'])
globals()['aoi_coords_w'] = list(globals()['aoi_geom_w'].coords)
globals()['aoi_coords_w'] = [(y,x) for x,y in globals()['aoi_coords_w']] 
globals()['aoi_centroid_w']= globals()['aoi_geom_w'].centroid
        
color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])]
folium.PolyLine(globals()['aoi_coords_w'],
                                        color=color[0],
                                        #fill_color=color[0],
                                        fill_opacity=0.2,
                                        weight=3).add_to(m)


globals()['wkt_str_t'] = testing_path_polygon
globals()['aoi_geom_t'] = wkt.loads(globals()['wkt_str_t'])
globals()['aoi_coords_t'] = list(globals()['aoi_geom_t'].coords)
globals()['aoi_coords_t'] = [(y,x) for x,y in globals()['aoi_coords_t']] 
globals()['aoi_centroid_t']= globals()['aoi_geom_t'].centroid
        
color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])]
folium.PolyLine(globals()['aoi_coords_t'],
                                        color=color[0],
                                        #fill_color=color[0],
                                        fill_opacity=0.2,
                                        weight=3).add_to(m)
m